In [1]:
import urllib.request as urllib2
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import Request, urlopen
import datetime
import bs4
import re 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
%matplotlib inline  
pd.set_option('display.max_columns', None)
from time import sleep
os.environ["PATH"] ="C:\dev\chromedriver;C:\dev;" + os.environ['PATH']
import warnings
warnings.filterwarnings('ignore')

In [10]:
usd_eth_price_spot = 1082.44
usd_eth_price_future = 1099.28
notional_usd = 1000

# buy btc spot
notional_eth = notional_usd/usd_eth_price_spot
# short btc fut for same notional
trade_profit_future = notional_eth*usd_eth_price_future
profit = trade_profit_future-notional_usd
y = profit/notional_usd
print("Profit: $", profit )
print("Yield: ",round(y*100,2), "%")
print("Equivalent annual yield: ",round(12*y*100,2), "%")


Profit: $ 15.55744429252411
Yield:  1.56 %
equivalent annual yield:  18.67 %


## KRAKEN

In [32]:
future_public_url = "https://futures.kraken.com/derivatives/api/v3"
endpoint = "/orderbook"
sym="FI_ETHUSD_210129".lower()
postUrl = "symbol=%s" % sym
url=future_public_url+endpoint+"?"+postUrl
public_key = "cJsIbb7hbL51/RwSAf4/EIHkGMMcDo3jzhakVv4hvgwPqDcO+b13FSP8"
private_key = "9sABLZBREQNSEDdI5QVIZhKRAfIDILXonW08eGv51O+CTkONahWoXGa0XLJ4i34nDWDNIJE9ioZS9CdsAgCp2F0v"

In [33]:
url

'https://futures.kraken.com/derivatives/api/v3/orderbook?symbol=fi_ethusd_210129'

In [36]:
def sign_message(endpoint, postData, nonce=""):
    # step 1: concatenate postData, nonce + endpoint                
    message = postData + nonce + endpoint

    # step 2: hash the result of step 1 with SHA256
    sha256_hash = hashlib.sha256()
    sha256_hash.update(message.encode('utf8'))
    hash_digest = sha256_hash.digest()

    # step 3: base64 decode apiPrivateKey
    secretDecoded = base64.b64decode(self.apiPrivateKey)

    # step 4: use result of step 3 to has the result of step 2 with HMAC-SHA512
    hmac_digest = hmac.new(secretDecoded, hash_digest, hashlib.sha512).digest()

    # step 5: base64 encode the result of step 4 and return
    return base64.b64encode(hmac_digest)

In [34]:
postData = postData = postUrl + postBody
#signature = sign_message(endpoint, postData)
#authentHeaders = {"APIKey": public_key, "Authent": signature}
request = urllib2.Request(url)#, authentHeaders)
request.get_method = lambda: "GET"

In [35]:
response = urllib2.urlopen(request, timeout=10)
response = response.read().decode("utf-8")
response

'{"result":"success","orderBook":{"bids":[[1113.85, 910], [1113.65, 629], [1113.55, 1460], [1113.5, 78908], [1113.4, 781], [1111.9, 196080], [1111.85, 167275], [1111, 64500], [1110.95, 45400], [1110.3, 180504], [1110.2, 1000], [1109.75, 3084], [1108.95, 58634], [1108.9, 38399], [1108.4, 6021], [1108.3, 90800], [1107.75, 276647], [1106.05, 40000], [1106, 11775], [1105.9, 472275], [1105, 29580], [1101.35, 327508], [1098.6, 166285], [1092.2, 60852], [1083.3, 77419], [1066.35, 78538], [1062, 60057], [1000, 200], [803.1, 100], [789.3, 2000], [768.7, 100], [738.2, 100], [731.75, 6000], [675.9, 15000], [636.5, 20000]],"asks":[[1116.1, 50], [1116.45, 76522], [1116.5, 910], [1117.2, 22700], [1118.05, 3584], [1118.2, 6021], [1118.25, 199394], [1119.2, 1000], [1119.25, 18634], [1119.3, 45400], [1119.5, 38399], [1120.2, 225200], [1120.35, 11775], [1121.25, 1460], [1121.55, 90800], [1122, 244609], [1123.2, 29580], [1123.95, 285677], [1124.05, 181600], [1124.45, 18634], [1128.85, 308874], [1133.9, 7

'{"result":"success","orderBook":{"bids":[[1117.35, 50], [1117.3, 910], [1117.05, 64427], [1116.6, 22700], [1115.9, 196080], [1115.85, 1460], [1115.35, 189371], [1115.15, 599], [1114.7, 500], [1113.9, 64000], [1113.85, 202363], [1113.65, 45400], [1112.8, 212680], [1112.35, 1695], [1112.3, 18634], [1112.25, 3084], [1111.15, 130800], [1111.1, 6021], [1110.85, 250379], [1109.4, 40000], [1109.35, 181600], [1108.35, 11775], [1108.15, 38399], [1106.95, 29580], [1103.25, 18634], [1096.25, 79486], [1096.2, 308874], [1091.9, 147651], [1083.3, 77419], [1066.35, 78538], [1062, 60057], [1000, 200], [803.1, 100], [789.3, 2000], [768.7, 100], [738.2, 100], [731.75, 6000], [675.9, 15000], [636.5, 20000]],"asks":[[1119.35, 910], [1119.4, 19199], [1119.6, 59727], [1120.05, 21800], [1121.2, 41483], [1121.25, 157030], [1121.6, 1960], [1121.95, 45400], [1122.4, 213033], [1122.85, 18634], [1123.4, 6021], [1124.05, 207031], [1124.95, 90800], [1125.25, 314024], [1126.05, 11775], [1127.5, 29580], [1127.55, 18